In [1]:
%load_ext autoreload
%autoreload 2
import sys
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42
mpl.rcParams['font.sans-serif'] = "Arial"
mpl.rcParams['font.family'] = "sans-serif"
import time
import glob
import re
import random
import torch

sys.path.insert(0, '/Data/user/panhailin/git_lab/npspy')
import npspy as nps
from npspy import npsc

In [2]:
all_peps = [
    '1A', '1C', '1D', '1E', '1F', '1G', '1H', '1I', '1K', '1L', '1M', '1N', '1P', '1Q', '1R', '1S', '1T', '1V', '1W', '1Y',

    '1pS', '1SMe', '3SMe', '1SAc', '3SAc', '1Soct', '3Soct', 
    '3I', '3L', '3dI', '3dL', '1D02', '1R02', '1Y02', '1W02',
           
]

In [3]:
stat_df = []
for pep in all_peps:
    clean_80_obj = nps.io.read_pickle(f"../../../03.results/classification_on_clean_data/GSXGS/{pep}/{pep}_valid80_clean_obj.pkl")
    clean_20_obj = nps.io.read_pickle(f"../../../03.results/classification_on_clean_data/GSXGS/{pep}/{pep}_valid20_clean_obj.pkl")
    stat_df.append([pep, len(clean_80_obj), len(clean_20_obj)])
stat_df = pd.DataFrame(stat_df, columns=['pep', 'v80_clean', 'v20_clean'])
stat_df = stat_df.set_index('pep')
stat_df

,v80_clean,v20_clean
pep,,
1A,30668,7681
1C,14898,3733
1D,50784,12592
1E,35908,8946
1F,21069,5257
1G,40300,10180
1H,15885,3922
1I,91304,22808
1K,85097,21295


In [5]:
stat_df.sort_values(by='v80_clean')

,v80_clean,v20_clean
pep,,
1Y,14527,3678
3SMe,14793,3654
1C,14898,3733
1H,15885,3922
3dL,16253,4030
1W,16723,4178
1SAc,17484,4381
1S,18057,4500
1M,19814,4912


In [6]:
valid_stat_df = pd.read_csv("../../../../opo/02.analyses/04.GSXGS/02.get_valid_reads/03.tables/read_combine_stat.csv", index_col=0)

In [7]:
valid_stat_df = valid_stat_df[['valid80', 'valid20']]

In [8]:
all_stat_df = pd.concat([valid_stat_df, stat_df], axis=1)
all_stat_df

,valid80,valid20,v80_clean,v20_clean
1A,44811,11203,30668,7681
1C,21061,5265,14898,3733
1D,68988,17247,50784,12592
1D02,28124,7031,20454,5090
1E,50370,12592,35908,8946
1F,30080,7520,21069,5257
1G,56394,14099,40300,10180
1H,23151,5788,15885,3922
1I,129730,32433,91304,22808
1K,119142,29786,85097,21295


In [9]:
all_stat_df['v80_clean_percent'] = all_stat_df['v80_clean'] / all_stat_df['valid80']
all_stat_df['v20_clean_percent'] = all_stat_df['v20_clean'] / all_stat_df['valid20']

In [10]:
all_stat_df.to_csv('../../../04.tables/classification/GSXGS/filter_stat.csv')

In [11]:
all_stat_df

,valid80,valid20,v80_clean,v20_clean,v80_clean_percent,v20_clean_percent
1A,44811,11203,30668,7681,0.684386,0.685620
1C,21061,5265,14898,3733,0.707374,0.709022
1D,68988,17247,50784,12592,0.736128,0.730098
1D02,28124,7031,20454,5090,0.727279,0.723937
1E,50370,12592,35908,8946,0.712885,0.710451
1F,30080,7520,21069,5257,0.700432,0.699069
1G,56394,14099,40300,10180,0.714615,0.722037
1H,23151,5788,15885,3922,0.686147,0.677609
1I,129730,32433,91304,22808,0.703800,0.703234
1K,119142,29786,85097,21295,0.714249,0.714933
